In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
import os
import math
import random
import glob
import requests
from ast import literal_eval
from multiprocessing import Process

In [3]:
import xgboost  as xgb
import lightgbm as lgb

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA,TruncatedSVD
from sklearn.ensemble import RandomForestClassifier,ExtraTreesClassifier,VotingClassifier
from sklearn.metrics import f1_score,accuracy_score

In [5]:
data_path = './DataSet/'
random_state = 42

In [6]:
train = pd.read_csv(f'{data_path}Processed_Train.csv')
test = pd.read_csv(f'{data_path}Processed_Test.csv')

In [7]:
sub = pd.read_csv(f'{data_path}Sample_submission.csv')

In [8]:
# lb = pd.read_csv(f'{data_path}lgb_data_int.csv')

In [9]:
train.columns.shape

(2365,)

In [10]:
target = train.Col2
train.drop(['Col2'],axis=1,inplace=True)

In [11]:
kfold = StratifiedKFold(n_splits=3,random_state=random_state)

In [12]:
# index = train.columns[(train.dtypes == float)].values.tolist()

In [13]:
# cols = ['unique']

In [14]:
# index.extend(cols)

In [15]:
# index.extend(lb[(lb.imp > 0)].col.values)

In [16]:
# index.remove('Col918')
# index.remove('Col923')

In [17]:
# train = train[set(index)]
# test = test[set(index)]

In [18]:
# len(index)

In [19]:
train.head()

,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,...,Col2391,Col2392,Col2393,Col2394,Col2395,Col2396,Col2397,Col1_1,Col1_2,Col1_3
0,9.293588,9.065239,9.665686,10.271793,8.570669,10.063160,9.461641,10.265213,9.117741,10.323243,...,1,5.370638,5.370638,4.234107,3.135494,3.135494,2.079442,324,338,52
1,12.462389,12.292086,12.136882,12.136882,12.280498,12.167137,12.059049,12.023941,12.212814,12.133187,...,0,3.737670,3.737670,3.663562,2.564949,2.564949,2.484907,326,342,124
2,16.496090,16.349255,16.213692,16.128357,16.749305,16.532874,15.487130,15.747948,16.052846,16.225451,...,0,5.937536,5.937536,5.739793,3.091042,3.091042,2.833213,325,631,92
3,13.419063,13.614755,13.429098,13.429098,13.624019,13.541838,12.787490,12.956692,12.972670,12.915474,...,0,6.555357,6.555357,6.469250,6.858565,6.858565,6.612041,325,656,74
4,-14.013331,-14.915909,-12.023501,-14.692468,14.764533,14.551021,15.596943,15.510490,15.060657,14.997903,...,0,7.174724,7.174724,7.100027,7.698936,7.698936,7.513164,325,603,19


In [20]:
# model_rf = RandomForestClassifier(n_estimators=100)
model_lg = lgb.LGBMClassifier(class_weigth='balanced')
model_xgb = xgb.XGBClassifier(base_score=0.4)

In [21]:
# for train_idx,test_idx in kfold.split(train,target):
#     x_train,y_train = train.iloc[train_idx],target.iloc[train_idx]
#     x_test,y_test = train.iloc[test_idx],target.iloc[test_idx]
#     model_lg.fit(x_train,y_train)
#     pred = model_lg.predict(x_test)
#     print(f1_score(y_test,pred))

In [22]:
# pred = model_lg.predict(test)

In [23]:
# model_vc = VotingClassifier(estimators=[('rf',model_rf),('etc',model_etc),('lgb',model_lg),('xgb',model_xgb)])
# model_vc.fit(train,target)

In [24]:
for train_idx,test_idx in kfold.split(train,target):
    x_train,y_train = train.iloc[train_idx],target.iloc[train_idx]
    x_test,y_test = train.iloc[test_idx],target.iloc[test_idx]
    model_lg.fit(x_train,y_train,eval_metric=accuracy_score)
    model_xgb.fit(x_train,y_train,eval_metric=accuracy_score)
    pred = np.argmax(model_lg.predict_proba(x_test) + model_xgb.predict_proba(x_test),axis=1)
#     pred = np.argmax(model_lg.predict_proba(x_test),axis=1)
    print(accuracy_score(y_test,pred))

0.9008731381612738
0.8977739726027397
0.8982876712328767


In [25]:
# pred = np.argmax(model_lg.predict_proba(test) + model_xgb.predict_proba(test),axis=1)
# pred = np.argmax(model_lg.predict_proba(test),axis=1)

In [26]:
sub.shape

(20442, 2)

In [27]:
pred = np.argmax(model_lg.predict_proba(test) + model_xgb.predict_proba(test),axis=1)

In [28]:
sub.Col2 = pred

In [29]:
sub.to_csv('output.csv',index=False)

In [30]:
len(pred[pred==0])

20034